# Compte-rendu du TP d'estimation statistique

**Elève 1** : Hadrien SALEM

**Elève 2** : Emilie SALEM

## Import des fonctions

In [ ]:
# Import des bibliothèques
import numpy as np
from scipy.stats import norm
from scipy.stats import uniform
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

In [ ]:
# Densité de probabilité d'un mélange de gaussiennes en un point
def gm_pdf(x, mu, sigma, p):
    #Initialisation de la variable de sortie
    resultat = 0.0
    #Contrôle de la cohérence des paramètres d'entrée
    #Le vecteur de moyenne doit avoir la même longueur que le vecteur p
    if len(mu) != len(p):
        print('Erreur de dimension sur la moyenne')
    # Le vecteur des écart-types doit avoir la même longueur que le vecteur p
    elif len(sigma) != len(p):
            print('Erreur de dimension sur les écarts-types')
    else:
    # Calcul de la valeur de la densité
        for i in range(0, len(p)):
            resultat = resultat + p[i] * norm.pdf(x, mu[i], sigma[i])
    return resultat

In [ ]:
# Génération de nombre aléatoire suivant un mélange de gaussiennes
def gm_rnd(mu, sigma, p):
    # Initialisation de la variable de sortie
    resultat = 0.0
    #Contrôle de la cohérence des paramètres d'entrée
    #Le vecteur de moyenne doit avoir la même longueur que le vecteur p
    if len(mu) != len(p):
        print('Erreur de dimension sur la moyenne')
    # Le vecteur des écart-types doit avoir la même longueur que le vecteur p
    elif len(sigma) != len(p):
            print('Erreur de dimension sur sur les écarts-types')
    else:
    #Génération de l'échantillon
    # On échantillonne suivant une loi uniforme sur [0,1]
        u = uniform.rvs(loc = 0.0, scale = 1.0, size = 1)
    # % Chaque test suivant permet de définir un intervalle sur lequel la
    # probabilité d'appartenance de la variable uniforme est égale à l'une des
    # probabilités définie dans le vecteur p. Lorsque u appartient à l'un de
    # ces intervalles, c'est équivalent à avoir générer une variable aléatoire
    # suivant l'un des éléments de p. Par exemple, pour le premier test
    # ci-dessous, la probabilité que u appartienne à l'intervalle [0,p[0][ est
    # égale à p[0] puisque u suit une loi uniforme. Donc si u appartient à
    # [0,p[0][ cela est équivalent à avoir tirer suivant l'événement de probabilité p[0].
        if u < p[0]: # On test si on a généré un événement de probabilité p[0]
            resultat = sigma[0] * norm.rvs(loc = 0, scale = 1, size = 1) + mu[0]
            # Pour générer suivant une loi normale quelconque, il suffit de multiplier
            # une variable normale centrée réduite (moyenne nulle et écart-type égal à 1)
            # par l'écart-type désité et d'additionner la moyenne désirée au produit précédent.
        for i in range(1, len(p)):
            if (u > np.sum(p[0:i])) and (u <= np.sum(p[0:i+1])): # On test si on a généré
                # un événement de probabilité p[i]
                resultat = sigma[i] * norm.rvs(loc = 0.0, scale = 1.0, size = 1) + mu[i]
                # Pour générer suivant une loi normale quelconque, il suffit de multiplier
                # une variable normale centrée réduite (moyenne nulle et écart-type égal à 1)
                # par l'écart-type désité et d'additionner la moyenne désirée au produit précédent.
    return resultat

In [ ]:
def EM_algorithm(nbMaxIterations, mu_0, sigma_0, alpha_0, donnees) :

    # Valeurs d'initialisation
    mu_em = mu_0
    sigma_em = sigma_0
    alpha_em = alpha_0 # la somme doit faire 1

    nbIteration = 1 #Initialisation de la variable d'arrêt
    nbComposante = len(alpha_em) #Nombre de composantes du mélange
    nbDonnees = len(donnees)  #Nombre de données
    p = np.zeros(shape=(nbComposante, nbDonnees))
    #Déclaration et initialisation de la matrice qui va contenir les probabilités
    #p(k|x,theta_courant)

    alpha_em_new = alpha_em
    sigma_em_carre_new = sigma_em
    mu_em_new = mu_em
    donneesP = np.zeros(shape=(nbDonnees))

    while nbIteration < nbMaxIterations:
        for n in range(0, nbDonnees, 1):
            for k in range(0, nbComposante, 1):
                p[k, n] = alpha_em[k] * norm.pdf(x = donnees[n], loc = mu_em[k], scale = sigma_em[k])
            p[:, n] = p[:, n] / np.sum(p[:, n])
        for k in range(0, nbComposante, 1):
            alpha_em_new[k] = np.sum(p[k,:]) / nbDonnees
            for n in range(0, nbDonnees, 1):
                donneesP[n] = donnees[n] * p[k, n]
            mu_em_new[k]  = np.sum(donneesP) / np.sum(p[k, :])
            for n in range(nbDonnees):
                donneesP[n] = ((donnees[n] - mu_em_new[k]) ** 2) * p[k, n]
            sigma_em_carre_new[k] = np.sum(donneesP) / np.sum(p[k, :])
        mu_em = mu_em_new
        sigma_em = np.sqrt(sigma_em_carre_new)
        alpha_em = alpha_em_new
        nbIteration = nbIteration + 1
    
    return np.array([mu_em, sigma_em, alpha_em])

## Partie 1 : test de l'algorithme EM sur des données synthétiques

Après tests sur les données synthétiques, on remarque que:
- Changer les valeurs initiales peut avoir une influence significative sur l'estimation. Prendre des valeurs "proches" des valeurs réelles permet d'obtenir une meilleure estimation. Il est donc préférable de déterminer un ordre de grandeur des paramètres recherchés avant même d'utiliser l'algorithme EM.
- Pour un bon choix de paramètres initiaux, le nombre d'itérations semble avoir peu d'influence sur l'estimation. Il ne faut toutefois pas choisir une valeur trop faible, sans quoi l'algorithme n'a "pas le temps" de converger.

## Partie 2 : Galaxies

Nous allons essayer de proposer une estimation de la distribution des valeurs contenues dans `galaxies.txt` sous forme de mélange de gaussiennes.
Il nous faut avant tout déterminer le nombre de gaussiennes du mélange, et pour chacune d'entre elles, une valeur d'initialisation pour :

- sa moyenne $\mu_0$
- son écart-type $\sigma_0$
- sa "probabilité" $\alpha_0$

Pour simplifier la visualisation des données et éviter d'éventuels effets dûs aux grandes valeurs prises par les vitesses, nous les centrons et les réduisons.
Dans les deux figures ci-dessous, nous affichons les données d'abord comme nuage de points, puis sous forme d'histogramme.

In [ ]:
# On importe les données
x_galaxies = np.loadtxt("galaxies.txt")
x_galaxies = (x_galaxies - np.mean(x_galaxies)) / np.std(x_galaxies) # données centrées réduites
nb_samples = len(x_galaxies)

In [ ]:
# Affichage des valeurs de chaque échantillon
plt.plot(x_galaxies, 'g.')
plt.title('Distribution des vitesses')
plt.xlabel('Indice')
plt.ylabel('Vitesse')
plt.grid()
plt.show()

In [ ]:
# Affichage de la distribution des valeurs sous forme d'histogramme
def plot_galaxy_hist():
    plt.hist(x_galaxies, bins = 30, density = True, edgecolor = "black")
    plt.title('Distribution des vitesses')
    plt.xlabel('Vitesse')
    plt.ylabel("Nombre d'échantillons")

plot_galaxy_hist()
plt.show()

## Commentaire
Dans un premier temps, nous considérerons les grandes tendances que nous pouvons voir sur ces deux courbes. Trois grands ensembles semblent se distinguer, nous choisirons donc un mélange de trois gaussiennes. Pour le choix des valeurs d'initialisation, nous calculons la moyenne et l'écart-type de trois ensembles, choisis pour l'instant "à l'oeil" en observant les courbes: 
- entre la première et la 8ème valeur
- entre la 8ème et la 76ème valeur
- et de la 76ème à la dernière valeur.

In [ ]:
# Choix des valeurs d'initialisation et du nombre d'itérations pour 3 gaussiennes

batches = np.split(x_galaxies, [8,76])
mu_0 = np.array([np.mean(batch) for batch in batches])
sigma_0 = np.array([np.std(batch) for batch in batches])
alpha_0 = np.array([0.2, 0.7, 0.1])
nb_iterations = 40

In [ ]:
# Exécution de l'algorithme EM pour 3 gaussiennes
resultat = EM_algorithm(nb_iterations, mu_0, sigma_0, alpha_0, x_galaxies)
mu_em, sigma_em, alpha_em = resultat

def print_parameters(mu, sigma, alpha):
    print('Les paramètres estimés sont : ')
    print('Moyennes des composantes du mélange', mu)
    print('Ecart type des composantes du mélange', sigma)
    print('Probabilités des composantes du mélange', alpha)
    
print_parameters(mu_em, sigma_em, alpha_em)

In [ ]:
# Affichage de la distribution estimée, superposée à la distribution empirique des valeurs

x = np.linspace(-3, 3, 10000)
def plot_distribution_comparison(estimation):
    plot_galaxy_hist()
    plt.plot(x, estimation, 'r-', label = 'Estimée')
    plt.legend(loc='upper left', shadow=True, fontsize='x-large')

estimation_3_components = gm_pdf(x, mu_em, sigma_em, alpha_em)    
plot_distribution_comparison(estimation_3_components)
plt.show()

## Commentaire
Au global, la distribution estimée suit les tendances de la distribution empirique. En revanche, pour les valeurs comprises entre -1 et 1, il semblerait que l'estimation pourrait être plus précise. En particulier, deux maxima locaux proches semblent se distinguer et ne sont pas différenciés par notre premier estimateur.

In [ ]:
# Choix des valeurs d'initialisation et du nombre d'itérations pour 4 gaussiennes

batches = np.split(x_galaxies, [8,40,50])
mu_0 = np.array([np.mean(batch) for batch in batches])
sigma_0 = np.array([np.std(batch) for batch in batches])
alpha_0 = np.array([0.2, 0.4, 0.3, 0.1])

nb_iterations = 40
resultat = EM_algorithm(nb_iterations, mu_0, sigma_0, alpha_0, x_galaxies)
mu_em, sigma_em, alpha_em = resultat

estimation_4_components = gm_pdf(x, mu_em, sigma_em, alpha_em)    
plot_distribution_comparison(estimation_4_components)
plt.show()

print_parameters(mu_em, sigma_em, alpha_em)


## Commentaire
Avec un mélange de 4 gaussiennes, les deux maxima locaux du milieu sont mieux représentés, mais on perd en qualité pour la prédiction des vitesses moins probables (notamment les grandes vitesses, pour lesquelles on avait un pic distinct avec 3 gaussiennes).

Nous pouvons essayer d'améliorer ce modèle en affinant le clustering des valeurs. Pour cela, nous faisons appel à la méthode des K-moyennes via `sklearn`.

In [ ]:
def kmeans_cluster(data, n_clusters):
    kmeans = KMeans(n_clusters = n_clusters, random_state=0).fit(data.reshape(-1,1))
    labeled_data = np.stack((kmeans.labels_, data), axis =1) # zip data with the label
    labeled_data = labeled_data[labeled_data[:,0].argsort()] # order by label
    return np.split(labeled_data[:,1], np.unique(labeled_data[:, 0], return_index = True)[1][1:]) # split values based on labels

In [ ]:
batches = kmeans_cluster(x_galaxies, 4)
mu_0 = np.array([np.mean(batch) for batch in batches])
sigma_0 = np.array([np.std(batch) for batch in batches])
alpha_0 = np.array([0.2, 0.4, 0.3, 0.1])

In [ ]:
nb_iterations = 40
resultat = EM_algorithm(nb_iterations, mu_0, sigma_0, alpha_0, x_galaxies)
mu_em, sigma_em, alpha_em = resultat

estimation_4_components = gm_pdf(x, mu_em, sigma_em, alpha_em)    
plot_distribution_comparison(estimation_4_components)
plt.show()

print_parameters(mu_em, sigma_em, alpha_em)

## Commentaire
En appliquant cette méthode de clustering, on remarque que l'estimation est plus proche de la distribution empirique dans les cas "peu probables": on récupère le pic autour de 2.75, et on diminue l'erreur sur les ensembles de valeurs dont la densité est nulle. En revanche, les deux pics autour de 0 sont moins bien différenciés: on retrouve une distribution plus similaire à notre cas à 3 gaussiennes, bien que plus précise.

Il nous faudrait donc une méthode pour choisir plus inteligemment le nombre de gaussiennes. Dans la suite, nous essayons d'utiliser le BIC (Bayesian Information Criterion) et AIC (Akaike Information Criterion) pour déterminer le nombre de gaussiennes.

In [ ]:
n = 20    
bics = []
aics = []
for i in range(1, n):
    
    gmm = GaussianMixture(n_components=i)
    gmm.fit(x_galaxies.reshape(-1,1))
    bics.append(gmm.bic(x_galaxies.reshape(-1,1)))
    aics.append(gmm.aic(x_galaxies.reshape(-1,1)))
    
    
plt.plot(np.arange(1,n),bics, label = 'BIC')
plt.plot(np.arange(1,n),aics, label = 'AIC')
plt.legend(loc='best')
plt.show()

print(np.argmin(bics), np.argmin(aics))
    

## Commentaire

Les critères BIC et AIC permettent de favoriser la parcimonie, c'est-à-dire un équilibre entre la qualité de l'estimation et le nombre de paramètres du modèle. Avec le calcul ci-dessus, on détermine que le nombre de composantes du mélange devrait être 2 selon le critère BIC, et plus de 10 selon le critère AIC. Le BIC pénalise davantage le nombre de paramètres du modèle, en comparaison avec l'AIC, ce qui explique en partie cette différence.

On notera qu'à chaque exécution de la cellule ci-dessus, les minimums de AIC et BIC sont susceptibles de changer (BIC prend les valeurs 2 et 4, AIC est très variable entre 12 et 18)

Réalisons l'estimation dans ces deux cas.

In [ ]:
def default_alpha_0(size):
    return np.full((1,size), 1/size)[0]

# Pour 2 gaussiennes
batches = kmeans_cluster(x_galaxies, 2)
mu_0 = np.array([np.mean(batch) for batch in batches])
sigma_0 = np.array([np.std(batch) for batch in batches])
alpha_0 = default_alpha_0(2)

mu_em, sigma_em, alpha_em = EM_algorithm(nb_iterations, mu_0, sigma_0, alpha_0, x_galaxies)

estimation = gm_pdf(x, mu_em, sigma_em, alpha_em)    
plot_distribution_comparison(estimation)
plt.show()

# Pour 10 gaussiennes
batches = kmeans_cluster(x_galaxies, 10)
mu_0 = np.array([np.mean(batch) for batch in batches])
sigma_0 = np.array([np.std(batch) for batch in batches])
alpha_0 = default_alpha_0(10)

mu_em, sigma_em, alpha_em = EM_algorithm(nb_iterations, mu_0, sigma_0, alpha_0, x_galaxies)

estimation = gm_pdf(x, mu_em, sigma_em, alpha_em)    
plot_distribution_comparison(estimation)
plt.show()


## Commentaire

Nous notons premièrement que nous avons limité la deuxième estimation à 10 gaussiennes, car au-delà, les résultats ne semblaient pas s'afficher.

On voit qu'en utilisant le BIC, le modèle estimé est simple. Seules les variations les plus distinctives sont représentées.
A l'inverse, l'utilisation de 10 gaussiennes conduit à un modèle très complexe qui essaie de tenir compte de toutes les variations apparentes dans les données.

On aura sans doute tendance à préférer un nombre intermédiaire de gaussiennes. Peut-être faudrait-il trouver un autre critère plus adapté au problème afin de déterminer le nombre de gaussiennes. Il faudrait également un critère pour déterminer l'efficacité du modèle sans connaissance du modèle initial.

In [ ]:
# Pour 5 gaussiennes
batches = kmeans_cluster(x_galaxies, 5)
mu_0 = np.array([np.mean(batch) for batch in batches])
sigma_0 = np.array([np.std(batch) for batch in batches])
alpha_0 = default_alpha_0(5)

mu_em, sigma_em, alpha_em = EM_algorithm(nb_iterations, mu_0, sigma_0, alpha_0, x_galaxies)

estimation = gm_pdf(x, mu_em, sigma_em, alpha_em)    
plot_distribution_comparison(estimation)
plt.show()